# Video Game Sales Webapp Project: creating the database

In [1]:
import pandas as pd
import sqlite3

In [2]:
# Create db connection
connection = sqlite3.connect('videogame.db')
cursor = connection.cursor()

## Database Schema

1. Video Games table

    - Game_ID (INTEGER), Title (VARCHAR), Year_of_Release (INTEGER), Genre (TEXT), Rating (CHAR(4)), Developer, Publisher
    - PK Game_ID
    - Unique Constraint Game_ID, Title, Year

2. Sales table

    - Game_ID, Platform, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales
    - PK Game_ID, Platform, Year_of_Release
    - FK Game_ID references Game_ID from Games Table

3. Reviews table

    - Review_ID (INTEGER), Game_ID (INTEGER), Critic_Score (REAL)
    - PK Review_ID
    - FK Game_ID references Game_ID from Games Table


In [3]:
# Read data
df = pd.read_csv('Video_Games.csv', index_col=0)
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
index,,,,,,,,,,,,,,,,
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.Year_of_Release = df.Year_of_Release.fillna(0).astype(int)
df.Year_of_Release.replace({0:pd.NA}, inplace=True)

In [7]:
df.shape

(16928, 16)

In [8]:
df.isnull().sum()

Name                  2
Platform              0
Year_of_Release     273
Genre                 2
Publisher            55
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8668
Critic_Count       8668
User_Score         6769
User_Count         9210
Developer          6688
Rating             6836
dtype: int64

## Handling of Missing Values

In [5]:
df.drop(df[df.Name.isnull()].index, axis=0, inplace=True)

In [6]:
df.Critic_Score.fillna(df.Critic_Score.median(), inplace=True)

In [7]:
df.Developer.fillna(df.Publisher, inplace=True)

## Creating Separate DataFrames for each Table in the database

In [8]:
games_table = df.drop_duplicates(subset=['Name', 'Year_of_Release'])[['Name', 'Year_of_Release', 'Genre', 'Rating', 'Developer', 'Publisher']]
games_table.insert(0, 'Game_ID', games_table.reset_index().index)

In [9]:
df = df.merge(games_table[['Game_ID', 'Name', 'Year_of_Release']], on=['Name', 'Year_of_Release'])

In [18]:
sales_table = df.drop_duplicates(subset=['Game_ID', 'Platform'])[['Game_ID', 'Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]
sales_table.insert(0, 'Sales_ID', sales_table.reset_index().index)

In [11]:
reviews_table = df.drop_duplicates(subset='Game_ID')[['Game_ID', 'Critic_Score']]
reviews_table.insert(0, 'Review_ID', reviews_table.reset_index().index)

## Populating the Database

In [12]:
games_table.head()

,Game_ID,Name,Year_of_Release,Genre,Rating,Developer,Publisher
index,,,,,,,
0,0,Wii Sports,2006,Sports,E,Nintendo,Nintendo
1,1,Super Mario Bros.,1985,Platform,NaN,Nintendo,Nintendo
2,2,Mario Kart Wii,2008,Racing,E,Nintendo,Nintendo
3,3,Wii Sports Resort,2009,Sports,E,Nintendo,Nintendo
4,4,Pokemon Red/Pokemon Blue,1996,Role-Playing,NaN,Nintendo,Nintendo


In [19]:
sales_table.head()

,Sales_ID,Game_ID,Platform,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,0,0,Wii,41.36,28.96,3.77,8.45,82.53
1,1,1,NES,29.08,3.58,6.81,0.77,40.24
2,2,2,Wii,15.68,12.76,3.79,3.29,35.52
3,3,3,Wii,15.61,10.93,3.28,2.95,32.77
4,4,4,GB,11.27,8.89,10.22,1.00,31.37


In [15]:
reviews_table.head()

,Review_ID,Game_ID,Critic_Score
0,0,0,76.0
1,1,1,71.0
2,2,2,82.0
3,3,3,80.0
4,4,4,71.0


In [22]:
cursor.execute("DROP TABLE IF EXISTS games")
#Create games table
cursor.execute('''CREATE TABLE games(
  game_id INTEGER NOT NULL,
  name VARCHAR(100) NOT NULL,
  year_of_release INTEGER,
  genre VARCHAR(50),
  rating CHAR(5),
  developer VARCHAR(50),
  publisher VARCHAR(50),
  PRIMARY KEY(game_id)
  CONSTRAINT unique_id_name_year UNIQUE (game_id, name, year_of_release)
  FOREIGN KEY (game_id) REFERENCES sales(game_id)
  FOREIGN KEY (game_id) REFERENCES reviews(game_id)
)''')
#load oris dataframe into sql
games_table.to_sql('games', connection, if_exists='append', index = False)

12430

In [23]:
# check top 5 entries and make sure everything looks good
cursor.execute(''' SELECT * FROM games LIMIT 5;''')
colnames = cursor.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

result_df = pd.DataFrame(cursor.fetchall(), columns=colnames_list)
result_df

,game_id,name,year_of_release,genre,rating,developer,publisher
0,0,Wii Sports,2006,Sports,E,Nintendo,Nintendo
1,1,Super Mario Bros.,1985,Platform,None,Nintendo,Nintendo
2,2,Mario Kart Wii,2008,Racing,E,Nintendo,Nintendo
3,3,Wii Sports Resort,2009,Sports,E,Nintendo,Nintendo
4,4,Pokemon Red/Pokemon Blue,1996,Role-Playing,None,Nintendo,Nintendo


In [24]:
cursor.execute("DROP TABLE IF EXISTS sales")
#Create games table
cursor.execute('''CREATE TABLE sales(
  sales_id INTEGER NOT NULL,
  game_id INTEGER NOT NULL,
  platform VARCHAR(6) NOT NULL,
  na_sales REAL NOT NULL,
  eu_sales REAL NOT NULL,
  jp_sales REAL NOT NULL,
  other_sales REAL NOT NULL,
  global_sales REAL NOT NULL,
  PRIMARY KEY(sales_id)
  CONSTRAINT unique_id_platform UNIQUE (sales_id, game_id, platform)
  FOREIGN KEY (game_id) REFERENCES games(game_id)
  FOREIGN KEY (game_ID) REFERENCES reviews(game_id)
)''')
#load oris dataframe into sql
sales_table.to_sql('sales', connection, if_exists='append', index = False)

16716

In [25]:
# check top 5 entries and make sure everything looks good
cursor.execute(''' SELECT * FROM sales LIMIT 5;''')
colnames = cursor.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

result_df = pd.DataFrame(cursor.fetchall(), columns=colnames_list)
result_df

,sales_id,game_id,platform,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,0,0,Wii,41.36,28.96,3.77,8.45,82.53
1,1,1,NES,29.08,3.58,6.81,0.77,40.24
2,2,2,Wii,15.68,12.76,3.79,3.29,35.52
3,3,3,Wii,15.61,10.93,3.28,2.95,32.77
4,4,4,GB,11.27,8.89,10.22,1.00,31.37


In [27]:
cursor.execute("DROP TABLE IF EXISTS reviews")
#Create games table
cursor.execute('''CREATE TABLE reviews(
  review_id INTEGER NOT NULL,
  game_id INTEGER NOT NULL,
  critic_score REAL,
  PRIMARY KEY(review_id)
  CONSTRAINT unique_id_game UNIQUE (review_id, game_id)
  FOREIGN KEY (game_id) REFERENCES games(game_id)
  FOREIGN KEY (game_id) REFERENCES sales(game_id)
)''')
#load oris dataframe into sql
reviews_table.to_sql('reviews', connection, if_exists='append', index = False)

12430

In [28]:
# check top 5 entries and make sure everything looks good
cursor.execute(''' SELECT * FROM reviews LIMIT 5;''')
colnames = cursor.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

result_df = pd.DataFrame(cursor.fetchall(), columns=colnames_list)
result_df

,review_id,game_id,critic_score
0,0,0,76.0
1,1,1,71.0
2,2,2,82.0
3,3,3,80.0
4,4,4,71.0


In [29]:
cursor.close()
connection.close()